<a href="https://colab.research.google.com/github/GavinHacker/ml_practice/blob/master/recbycontent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install pymysql

    100% |████████████████████████████████| 51kB 2.0MB/s 


In [0]:
import pandas as pd
import pymysql
import pymysql.cursors
from functools import reduce
import numpy as np
import pandas as pd
import uuid
import datetime
from pyfm import pylibfm
from sklearn.feature_extraction import DictVectorizer
from sklearn.metrics.pairwise import pairwise_distances
np.set_printoptions(precision=3)
np.set_printoptions(suppress=True)

In [0]:
def get_connection():
    return pymysql.connect(host='rm-2zeqqm6994abi7b6dqo.mysql.rds.aliyuncs.com',
                               user='noone',
                               password='Huawei12#$',
                               db='recsys',
                               port=3306,
                               charset ='utf8',
                               use_unicode=True)

In [0]:
df = pd.read_sql_query("select * from movie", get_connection())

In [130]:
df.shape

(6629, 10)

In [0]:
df_exp = df.head(10)

In [132]:
df_exp

,id,name,ADD_TIME,type,actors,region,director,trait,rat,enable
0,10001432,颠倒的帕特玛,2018-02-07 20:46:08,剧情|冒险|科幻|爱情,藤井雪代|冈本信彦|大畑伸太郎|福松进纱|加藤将之,日本,吉浦康裕,青春,7.9,1
1,10047547,饥饿游戏3：嘲笑鸟(下),2018-02-07 20:27:08,奇幻|惊悚|爱情|战争|剧情|科幻|冒险|动作,詹妮弗·劳伦斯|乔什·哈切森|利亚姆·海姆斯沃斯|伍迪·哈里森|伊丽莎白·班克斯,美国,弗朗西斯·劳伦斯,青春,6.2,1
2,10342838,弯刀杀戮,2018-02-07 20:42:15,喜剧|犯罪|惊悚|动作,丹尼·特雷霍|米歇尔·罗德里格兹|索菲娅·维加拉|艾梅柏·希尔德|Lady Gaga,俄罗斯,罗伯特·罗德里格兹,搞笑,5.7,1
3,10355329,巴菲的奇妙命运,2018-02-07 20:44:01,剧情|喜剧|音乐|爱情,佩丽冉卡·曹帕拉|兰比尔·卡普尔|伊利亚娜·狄克鲁兹|Akash Khurana|沙鲁巴·舒克拉,印度,阿努拉格·巴苏,感人|经典|励志|搞笑|文艺,8.3,1
4,10428476,小屁孩日记3,2018-02-07 20:47:00,剧情|喜剧,扎克瑞·戈登|瑞秋·哈里斯|戴文·博斯蒂克|罗伯特·卡普荣|佩顿·李斯特,加拿大,大卫·博维斯,青春|搞笑,6.7,1
5,10430817,隐墙,2018-02-07 21:03:06,剧情|奇幻|悬疑|科幻,马蒂娜·格德克|Karlheinz Hackl|Wolfgang M|Bauer|Ulrik...,德国,Julian Pölsler,文艺,7.4,1
6,10432911,流感,2018-02-07 20:28:33,惊悚|爱情|剧情|悬疑|灾难|动作,张赫|秀爱|朴敏荷|柳海真|李熙俊,韩国,金成洙,感人,7.8,1
7,10437779,新世界,2018-02-07 20:24:56,惊悚|爱情|剧情|犯罪|悬疑|传记|历史|动作,李政宰|崔岷植|黄政民|宋智孝|朴圣雄,韩国,朴勋政,经典|黑帮|文艺,8.6,1
8,10437865,青春感恩记《父亲》之《父子篇》,2018-02-07 20:51:58,剧情,肖央|吴坚|李佳蔚,中国大陆,肖央,感人,6.3,1
9,10438426,浮城谜事,2018-02-07 20:26:53,剧情|犯罪|悬疑|爱情,郝蕾|秦昊|齐溪|朱亚文|祖锋,法国,娄烨,女性|文艺,7.3,1


In [0]:
def get_dim_dict(df, dim_name):
  type_list = list(map(lambda x:x.split('|') ,df[dim_name]))
  type_list = [x for l in type_list for x in l]
  def reduce_func(x, y):
    for i in x:
      if i[0] == y[0][0]:
        x.remove(i)
        x.append(((i[0],i[1] + 1)))
        return x
    x.append(y[0])
    return x
  l = filter(lambda x:x != None, map(lambda x:[(x, 1)], type_list))
  type_zip = reduce(reduce_func, list(l))
  #type_list = sorted(list(set(type_list)))
  #type_zip = zip(list(range(len(type_list))), type_list)
  print(len(type_zip))
  print(type_zip)
  type_dict = {}
  for i in type_zip:
    type_dict[i[0]] = i[1]
  return type_dict

In [314]:
type_dict = get_dim_dict(df, 'type')
actors_dict = get_dim_dict(df, 'actors')
director_dict = get_dim_dict(df, 'director')
trait_dict = get_dim_dict(df, 'trait')

21
[('西部', 54), ('同性', 318), ('恐怖', 463), ('情色', 289), ('战争', 447), ('灾难', 114), ('音乐', 425), ('歌舞', 225), ('奇幻', 1099), ('传记', 478), ('悬疑', 1027), ('科幻', 817), ('武侠', 150), ('历史', 466), ('爱情', 3312), ('惊悚', 1172), ('剧情', 5191), ('犯罪', 1350), ('喜剧', 2856), ('冒险', 974), ('动作', 1661)]
12558
[('藤井雪代', 1), ('大畑伸太郎', 1), ('福松进纱', 1), ('加藤将之', 1), ('丹尼·特雷霍', 1), ('兰比尔·卡普尔', 1), ('伊利亚娜·狄克鲁兹', 1), ('Akash Khurana', 1), ('Karlheinz Hackl', 1), ('Wolfgang M', 1), ('Bauer', 1), ('Ulrike Beimpold', 1), ('Julia Gschnitzer', 1), ('朴敏荷', 1), ('李熙俊', 1), ('吴坚', 1), ('李佳蔚', 1), ('祖锋', 1), ('萨尔瓦多·斯特里亚诺', 1), ('柯西莫·雷加', 1), ('安东尼奥·弗拉斯加', 1), ('胡安·达里奥·伯内蒂', 1), ('文森特·盖洛', 1), ('Adam Savage', 1), ('Jamie Hyneman', 1), ('卡文·英玛诺泰', 1), ('露丝·莱斯利', 1), ('朱莉·恩格尔布雷希特', 1), ('艾玛·福尔曼', 1), ('妮珂莱特·皮尔妮 ', 1), ('姜峰', 1), ('尼玛颂宋', 1), ('雷茜', 1), ('游大庆', 1), ('米洛·文堤米利亚', 1), ('宫藤官九郎', 1), ('胡夏', 1), ('大左', 1), ('徐正超', 1), ('迈克尔·C·豪尔', 1), ('黛西·贝文', 1), ('大卫·沃肖夫斯基', 1), ('张岫云', 1), ('陈文彬', 1), ('罗贝尔·普拉尼奥尔', 1), ('乔纳森·科恩

In [315]:
#print(type_dict)
print(actors_dict)
print(director_dict)
#print(trait_dict)

{'藤井雪代': 1, '大畑伸太郎': 1, '福松进纱': 1, '加藤将之': 1, '丹尼·特雷霍': 1, '兰比尔·卡普尔': 1, '伊利亚娜·狄克鲁兹': 1, 'Akash Khurana': 1, 'Karlheinz Hackl': 1, 'Wolfgang M': 1, 'Bauer': 1, 'Ulrike Beimpold': 1, 'Julia Gschnitzer': 1, '朴敏荷': 1, '李熙俊': 1, '吴坚': 1, '李佳蔚': 1, '祖锋': 1, '萨尔瓦多·斯特里亚诺': 1, '柯西莫·雷加': 1, '安东尼奥·弗拉斯加': 1, '胡安·达里奥·伯内蒂': 1, '文森特·盖洛': 1, 'Adam Savage': 1, 'Jamie Hyneman': 1, '卡文·英玛诺泰': 1, '露丝·莱斯利': 1, '朱莉·恩格尔布雷希特': 1, '艾玛·福尔曼': 1, '妮珂莱特·皮尔妮 ': 1, '姜峰': 1, '尼玛颂宋': 1, '雷茜': 1, '游大庆': 1, '米洛·文堤米利亚': 1, '宫藤官九郎': 1, '胡夏': 1, '大左': 1, '徐正超': 1, '迈克尔·C·豪尔': 1, '黛西·贝文': 1, '大卫·沃肖夫斯基': 1, '张岫云': 1, '陈文彬': 1, '罗贝尔·普拉尼奥尔': 1, '乔纳森·科恩': 1, '薇迪亚·巴兰': 1, '帕拉姆巴拉塔·查特吉': 1, '特里蒂曼·查特吉': 1, '萨斯瓦塔·查特吉': 1, '丁佳明': 1, '简嫚书': 1, '吴卓羲': 1, '孙祖扬': 1, '尹雅宁': 1, '吴冰滨': 1, '赤羽根健治': 1, '李贤镇': 1, '李素晶': 1, '李茂生': 1, '托尼·丹扎': 1, '林·沙烨': 1, '赵敏修': 1, '伍基洪': 1, '姜恩珍': 1, 'Anna Chuancheun': 1, '曹议文': 1, '葛蕾': 1, '阿加塔·布泽克': 1, '盖·莱恩': 1, '朴晓英': 1, '袁牧女': 1, '王烈': 1, '卡里·希耶塔拉赫蒂': 1, '沃科·霍瓦塔': 1, '尤哈·韦约宁': 1, '埃里克·卡尔森': 1, '艾丽娜·克尼赫蒂

In [316]:
director_dict['吉姆·弗尔']

2

In [317]:
actors_dict['马克·莱昂纳蒂']

2

In [0]:
df_ = df.drop(['ADD_TIME', 'enable', 'rat', 'id', 'name'], axis=1)

In [319]:
df_.head()

,type,actors,region,director,trait
0,剧情|冒险|科幻|爱情,藤井雪代|冈本信彦|大畑伸太郎|福松进纱|加藤将之,日本,吉浦康裕,青春
1,奇幻|惊悚|爱情|战争|剧情|科幻|冒险|动作,詹妮弗·劳伦斯|乔什·哈切森|利亚姆·海姆斯沃斯|伍迪·哈里森|伊丽莎白·班克斯,美国,弗朗西斯·劳伦斯,青春
2,喜剧|犯罪|惊悚|动作,丹尼·特雷霍|米歇尔·罗德里格兹|索菲娅·维加拉|艾梅柏·希尔德|Lady Gaga,俄罗斯,罗伯特·罗德里格兹,搞笑
3,剧情|喜剧|音乐|爱情,佩丽冉卡·曹帕拉|兰比尔·卡普尔|伊利亚娜·狄克鲁兹|Akash Khurana|沙鲁巴·舒克拉,印度,阿努拉格·巴苏,感人|经典|励志|搞笑|文艺
4,剧情|喜剧,扎克瑞·戈登|瑞秋·哈里斯|戴文·博斯蒂克|罗伯特·卡普荣|佩顿·李斯特,加拿大,大卫·博维斯,青春|搞笑


In [0]:
movie_dict_list = []
for i in df_.index:
  movie_dict = {}
  #type
  for s_type in df_.iloc[i]['type'].split('|'):
    movie_dict[s_type] = 1
  #actors
  for s_actor in df_.iloc[i]['actors'].split('|'):
    if actors_dict[s_actor] < 2:
      movie_dict['other_actor'] = 1
    else:
      movie_dict[s_actor] = 1
  #regios
  movie_dict[df_.iloc[i]['region']] = 1
  #director
  for s_director in df_.iloc[i]['director'].split('|'):
    if director_dict[s_director] < 2:
      movie_dict['other_director'] = 1
    else:
      movie_dict[s_director] = 1
  #trait
  for s_trait in df_.iloc[i]['trait'].split('|'):
    movie_dict[s_trait] = 1
  movie_dict_list.append(movie_dict)

In [0]:
v = DictVectorizer()
X = v.fit_transform(movie_dict_list)

In [348]:
X.shape

(6629, 5855)

In [349]:
pd.DataFrame(data=X.toarray(), columns=v.feature_names_).T

,0,1,2,3,4,5,6,7,8,9,...,6619,6620,6621,6622,6623,6624,6625,6626,6627,6628
50分,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Aaron Seltzer,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Aleksandr Voytinskiy,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Allan Brocka,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Andrei Shchetinin,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Anna Aleksakhina,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Arcy,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Ben Washam,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Bernhard Schütz,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [0]:
item_similarity = pairwise_distances(X, metric='cosine')

In [351]:
item_similarity.shape

(6629, 6629)

In [365]:
index = 0
_max_index = 0
_max = 1
for i in item_similarity[1]:
  if i < _max and i != 0:
    _max = i
    _max_index = index
  index = index + 1
print(_max_index, _max)

6526 0.03175416344814619


In [366]:
index_of_sim = 6526
df.iloc[index_of_sim]
movie_dict_list[index_of_sim]
df_106 = pd.DataFrame(data=X.todense()[index_of_sim], columns=v.feature_names_)
df_0 = pd.DataFrame(data=X.todense()[0], columns=v.feature_names_)
df_diff = pd.concat([df_0, df_106], axis=0, ignore_index=True)
#df_diff.reindex(['0','1'])
df_diff = df_diff.T
df_diff[(df_diff[0] != 0) | (df_diff[1] != 0)]

,0,1
other_actor,1.0,0.0
other_director,1.0,0.0
乔什·哈切森,0.0,1.0
伊丽莎白·班克斯,0.0,1.0
伍迪·哈里森,0.0,1.0
冈本信彦,1.0,0.0
冒险,1.0,1.0
利亚姆·海姆斯沃斯,0.0,1.0
剧情,1.0,1.0
动作,0.0,1.0


In [367]:
df.iloc[1]
#movie_dict_list[0]

id                                          10047547
name                                    饥饿游戏3：嘲笑鸟(下)
ADD_TIME                         2018-02-07 20:27:08
type                         奇幻|惊悚|爱情|战争|剧情|科幻|冒险|动作
actors      詹妮弗·劳伦斯|乔什·哈切森|利亚姆·海姆斯沃斯|伍迪·哈里森|伊丽莎白·班克斯
region                                            美国
director                                    弗朗西斯·劳伦斯
trait                                             青春
rat                                              6.2
enable                                             1
Name: 1, dtype: object

In [368]:
df.iloc[index_of_sim]

id                                           6874456
name                                      饥饿游戏2：星火燎原
ADD_TIME                         2018-02-07 20:22:25
type                            奇幻|惊悚|爱情|剧情|科幻|冒险|动作
actors      詹妮弗·劳伦斯|乔什·哈切森|利亚姆·海姆斯沃斯|伍迪·哈里森|伊丽莎白·班克斯
region                                            美国
director                                    弗朗西斯·劳伦斯
trait                                             青春
rat                                              7.3
enable                                             1
Name: 6526, dtype: object